In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field
from typing import List

# === Pydantic 모델 정의 ===
class StudentIntro(BaseModel):
    name: str = Field(description="학생의 이름")
    age: int = Field(description="학생의 나이 (숫자)")
    major: str = Field(description="전공")
    hobbies: List[str] = Field(description="취미 목록 (문자열 리스트)")
    goal: str = Field(description="장기적인 목표 또는 꿈")

# 출력 파서
parser = PydanticOutputParser(pydantic_object=StudentIntro)

# === 프롬프트 템플릿 ===
template = """
다음 학생의 자기소개를 읽고, 이름, 나이, 전공, 취미, 목표를 각각 추출해 주세요.
각 항목은 다음 형식에 맞게 출력해 주세요:

{format_instructions}

자기소개:
{text}
"""

# 프롬프트 구성
prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# === 모델 초기화 ===
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

# === 테스트 입력 ===
intro_text = [
    "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다.",
    "제 이름은 박지연이고, 올해 20세입니다. 경영학과에 다니고 있고, 독서와 여행, 요리를 즐깁니다. 미래에는 글로벌 기업의 CEO가 되고 싶어요.",
    "이수진입니다. 나이는 23살이고 미술을 전공하고 있습니다. 그림 그리기, 사진 찍기, 전시회 관람이 취미예요. 유명한 작가가 되어 많은 사람들에게 감동을 주고 싶습니다.",
    "안녕하세요~ 저는 최진우예요. 21살이고 기계공학과 학생입니다. 축구, 농구 같은 운동을 좋아하고 음악 듣기도 즐겨해요. 앞으로 로봇 엔지니어가 되는 게 꿈이에요."
]

# === 체인 구성 ===
chain = prompt | model | parser

# === 반복 실행 ===
for idx, text in enumerate(intro_text, start=1):
    output = chain.invoke({"text": text})
    print(f"\n=== 학생 {idx} 정보 ===")
    print(f"이름: {output.name}")
    print(f"나이: {output.age}")
    print(f"전공: {output.major}")
    print(f"취미: {', '.join(output.hobbies)}")
    print(f"목표: {output.goal}")



=== 학생 1 정보 ===
이름: 김민수
나이: 22
전공: 컴퓨터공학
취미: 게임하기, 영화보기, 코딩
목표: 훌륭한 개발자가 되는 것

=== 학생 2 정보 ===
이름: 박지연
나이: 20
전공: 경영학과
취미: 독서, 여행, 요리
목표: 글로벌 기업의 CEO가 되고 싶어요

=== 학생 3 정보 ===
이름: 이수진
나이: 23
전공: 미술
취미: 그림 그리기, 사진 찍기, 전시회 관람
목표: 유명한 작가가 되어 많은 사람들에게 감동을 주고 싶습니다.

=== 학생 4 정보 ===
이름: 최진우
나이: 21
전공: 기계공학과
취미: 축구, 농구, 음악 듣기
목표: 로봇 엔지니어
